In [ ]:
import pymysql

ENDPOINT = '10.10.211.59'
PORT = 3306
USERNAME = 'read_only'
DBNAME = 'CAPC_ACCURACY_REPORT'
PASSWORD = 'P@ssw0rd@123' # Change this based on your own password
CURSORCLASS = pymysql.cursors.DictCursor

In [ ]:
def initiate_local_connection():
    try:
        connection = pymysql.connect(host=ENDPOINT,
                                     port=PORT,
                                     user=USERNAME,
                                     passwd=PASSWORD,
                                     db=DBNAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Local Connection Successful')
    except Exception as e:
        print(f'[+] Local Connection Failed: {e}')
        connection = None

    return connection

In [ ]:
#  Call function and return the connection object
connection = initiate_local_connection()

In [ ]:
query = '''select * from overall_data where month in ('2023-07') and client_code='CHS';'''

In [ ]:
df_query = pd.read_sql(query, con=connection)

In [ ]:
df_query

In [ ]:
df_query.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx')

In [ ]:
df = df_query.copy()

In [119]:
from collections import OrderedDict
import pandas as pd
def get_accuracy(data_df, codetype):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df[codetype+'_accepted_count'].sum()
    reject = data_df[codetype+'_rejected_count'].sum()
    add = data_df[codetype+'_added_count'].sum()
    
    d = OrderedDict()
    # d['facility_name'] = data_df['facility_name'].iloc[0]
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [24]:
df.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True])

,encounters,eandm_accepted,eandm_rejected,eandm_added,eandm_billed,precision,recall,f1
facility_code,,,,,,,,
1475.0,1004.0,643.0,271.0,121.0,764.0,0.703501,0.841623,0.766389
1523.0,409.0,148.0,95.0,90.0,238.0,0.609053,0.621849,0.615385
1570.0,413.0,166.0,126.0,124.0,290.0,0.568493,0.572414,0.570447
1575.0,316.0,132.0,151.0,119.0,251.0,0.466431,0.525896,0.494382
1739.0,163.0,46.0,24.0,20.0,66.0,0.657143,0.696970,0.676471
1902.0,1735.0,932.0,486.0,430.0,1362.0,0.657264,0.684288,0.670504
1930.0,527.0,346.0,118.0,96.0,442.0,0.745690,0.782805,0.763797
2245.0,179.0,84.0,57.0,49.0,133.0,0.595745,0.631579,0.613139
2445.0,77.0,47.0,19.0,12.0,59.0,0.712121,0.796610,0.752000


In [ ]:
df.enc_doc_id = df.enc_doc_id.astype('str')

In [ ]:
df.shape

In [ ]:
df_single_docs = df[~df.enc_doc_id.str.contains(',')]

In [ ]:
df[df.enc_visit_type.str.contains('RECHECK')][cols_to_use]

In [ ]:
df_single_docs.shape

In [ ]:
df_9954 = df[df['facility_code'] == '9954']

In [ ]:
df_9954.shape

In [ ]:
df_9954.groupby(['enc_visit_type']).apply(get_accuracy, codetype='eandm').sort_values(['eandm_rejected'],ascending=[False])

In [ ]:
df_9954.enc_visit_type = df_9954.enc_visit_type.astype('str')

In [ ]:
df.columns

In [ ]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'eandm_suggested', 'eandm_final', 'cpt_suggested', 'cpt_final']

In [ ]:
df_9954[df_9954.enc_visit_type.str.contains('N/A')][cols_to_use]

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx', index_col=False)

In [184]:
df.columns

Index(['Unnamed: 0', 'id', 'encounter_id', 'physician_id', 'service_date',
       'processing_date', 'admission_date', 'user_id', 'process_id',
       'last_coding_date', 'month', 'enc_visit_type', 'enc_place_of_service',
       'account_id', 'client_code', 'facility_code', 'account_number',
       'date_of_birth', 'gender', 'point_of_care', 'acc_visit_type',
       'acc_place_of_service', 'patient_class', 'patient_type', 'service_line',
       'financial_class', 'payor', 'enc_doc_id', 'enc_doc_type', 'ref_doc_id',
       'ref_doc_type', 'ref_codes', 'cm_final', 'cm_suggested', 'cm_accepted',
       'cm_rejected', 'cm_added', 'cm_accepted_count', 'cm_rejected_count',
       'cm_added_count', 'cpt_final', 'cpt_suggested', 'cpt_accepted',
       'cpt_rejected', 'cpt_added', 'cpt_accepted_count', 'cpt_rejected_count',
       'cpt_added_count', 'eandm_final', 'eandm_suggested', 'eandm_accepted',
       'eandm_rejected', 'eandm_added', 'eandm_accepted_count',
       'eandm_rejected_count', 

In [3]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'cpt_final', 'cpt_suggested','eandm_final', 'eandm_suggested','local_suggested_ruleEngine','eandm_accepted_count',
       'eandm_rejected_count', 'eandm_added_count', 'local_suggested_xray_used']

In [4]:
df_copy = df.copy()

In [5]:
df_copy = df_copy[cols_to_use]

In [6]:
df_copy.local_suggested_ruleEngine = df_copy.local_suggested_ruleEngine.astype('str')

In [7]:
df_copy = df_copy[~df_copy.local_suggested_ruleEngine.str.contains('vlookup_null')]

In [190]:
# df_copy.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/local_JULY.xlsx')

In [8]:
import re
for i,r in df_copy.iterrows():
    df_copy.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    df_copy.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

In [9]:
df_copy

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,local_suggested_ruleEngine,eandm_accepted_count,eandm_rejected_count,eandm_added_count,local_suggested_xray_used,clean_suggested,clean_final
0,8235901,2023-07-06 11:00:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095114,85610,"{85610=[QW], 1036F=[]}",NaN,{},99212,0,0,0,99212,,
1,8256124,2023-07-06 13:30:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095124,85610,"{1170F=[], 1160F=[], 85610=[QW], 1159F=[], 103...",NaN,{},99213,0,0,0,99213,,
2,8256117,2023-07-06 13:15:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8803379,85610,"{85610=[QW], 1035F=[], 1034F=[]}",NaN,{},99212,0,0,0,99212,,
3,8229493,2023-07-05 14:00:00,*Hosp Follow-up 30,Office or Other Outpatient Visit,3038,8057412,"3008F, 93040, 3077F, 1036F, 3078F","{3008F=[], 93040=[], 3077F=[], 1036F=[], 3078F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214
4,8234612,2023-07-06 09:30:00,*Follow-up 15,Office or Other Outpatient Visit,3038,8057430,"1160F, 93040, 1159F, 1126F, 1036F","{1160F=[], 93040=[], 1159F=[], 1126F=[], 1036F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414,8228747,2023-07-05 14:00:00,*Follow-up 15,OV,1523,4060723,NaN,{81001=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213
8415,8082306,2023-06-20 13:00:00,uroflow/PVR,OV,1523,4329261,"1160F, 1159F, 1036F","{1160F=[], 1159F=[], 1036F=[]}",99214,{99214=[25]},99214,1,0,0,99214,99214,99214
8416,8213382,2023-07-05 13:30:00,ER FOLLOW-UP,OV,1523,4351334,"1160F, 1159F, 1036F","{3008F=[], 1160F=[], 1159F=[], 87210=[QW], 103...",99213,{99213=[]},99213,1,0,0,99213,99213,99213
8417,8113030,2023-06-30 11:30:00,ESTABLISHED PATIENT,OV,1523,4349846,81025,{81025=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213


In [10]:
def get_counts(x):
    
    suggested = str(x['local_suggested_xray_used'])
    final = str(x['clean_final'])
    
    if suggested != '':
        suggested_set = set(suggested.split(', '))
        final_set = set(final.split(', '))
        if '9' not in final:
            final_set.clear()     
    
        accepted = len(suggested_set.intersection(final_set))
        rejected = len(suggested_set - final_set)
        added = len(final_set - suggested_set)
        
        if (rejected >= 1 or added >= 1):
            print('rejected',rejected)
            print('added',added)
            print('suggested_set',suggested_set,'\t final_set',final_set)
        return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [11]:
count_df =  df_copy.apply(get_counts, axis=1)

rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99213'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99204'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99212'} 	 final_set set()
rejected 1
added 0
suggested_set {'99213'} 	 final_set set()
rejected 1
added 0
sugge

In [12]:
df2 = pd.concat([df_copy, count_df], axis = 1)

In [13]:
df2

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,local_suggested_ruleEngine,eandm_accepted_count,eandm_rejected_count,eandm_added_count,local_suggested_xray_used,clean_suggested,clean_final,accepted,rejected,added
0,8235901,2023-07-06 11:00:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095114,85610,"{85610=[QW], 1036F=[]}",NaN,{},99212,0,0,0,99212,,,0,1,0
1,8256124,2023-07-06 13:30:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095124,85610,"{1170F=[], 1160F=[], 85610=[QW], 1159F=[], 103...",NaN,{},99213,0,0,0,99213,,,0,1,0
2,8256117,2023-07-06 13:15:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8803379,85610,"{85610=[QW], 1035F=[], 1034F=[]}",NaN,{},99212,0,0,0,99212,,,0,1,0
3,8229493,2023-07-05 14:00:00,*Hosp Follow-up 30,Office or Other Outpatient Visit,3038,8057412,"3008F, 93040, 3077F, 1036F, 3078F","{3008F=[], 93040=[], 3077F=[], 1036F=[], 3078F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,1,0,0
4,8234612,2023-07-06 09:30:00,*Follow-up 15,Office or Other Outpatient Visit,3038,8057430,"1160F, 93040, 1159F, 1126F, 1036F","{1160F=[], 93040=[], 1159F=[], 1126F=[], 1036F...",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414,8228747,2023-07-05 14:00:00,*Follow-up 15,OV,1523,4060723,NaN,{81001=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213,1,0,0
8415,8082306,2023-06-20 13:00:00,uroflow/PVR,OV,1523,4329261,"1160F, 1159F, 1036F","{1160F=[], 1159F=[], 1036F=[]}",99214,{99214=[25]},99214,1,0,0,99214,99214,99214,1,0,0
8416,8213382,2023-07-05 13:30:00,ER FOLLOW-UP,OV,1523,4351334,"1160F, 1159F, 1036F","{3008F=[], 1160F=[], 1159F=[], 87210=[QW], 103...",99213,{99213=[]},99213,1,0,0,99213,99213,99213,1,0,0
8417,8113030,2023-06-30 11:30:00,ESTABLISHED PATIENT,OV,1523,4349846,81025,{81025=[]},99213,{99213=[]},99213,1,0,0,99213,99213,99213,1,0,0


In [14]:
df2.eandm_rejected_count.sum()

2272

In [15]:
df2.rejected.sum()

3834

In [16]:
from collections import OrderedDict
def get_accuracy_new(data_df):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df['accepted'].sum()
    reject = data_df['rejected'].sum()
    add = data_df['added'].sum()
    
    d = OrderedDict()
    d['encounters'] = data_df.shape[0]
    
    d['New'+'_accepted'] = accept
    d['New'+'_rejected'] = reject
    d['New'+'_added'] = add
    
    d['New'+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
df2.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_local.xlsx')

In [148]:
df2.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_with_rule.xlsx')

In [152]:
new_df = df2.copy()

In [153]:

new_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8419 entries, 0 to 8418
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   encounter_id                8419 non-null   int64         
 1   service_date                8419 non-null   datetime64[ns]
 2   enc_visit_type              8309 non-null   object        
 3   enc_place_of_service        8419 non-null   object        
 4   facility_code               8419 non-null   int64         
 5   account_number              8419 non-null   int64         
 6   cpt_final                   6633 non-null   object        
 7   cpt_suggested               8419 non-null   object        
 8   eandm_final                 6318 non-null   object        
 9   eandm_suggested             8419 non-null   object        
 10  local_suggested_ruleEngine  8419 non-null   object        
 11  eandm_accepted_count        8419 non-null   int64       

In [155]:
for i, r in new_df.iterrows():
    if '99024' in str(r['cpt_suggested']) or '0502F' in str(r['cpt_suggested']):
        new_df.at[i, 'accepted'] = 1

In [156]:
new_df

,encounter_id,service_date,enc_visit_type,enc_place_of_service,facility_code,account_number,cpt_final,cpt_suggested,eandm_final,eandm_suggested,local_suggested_ruleEngine,eandm_accepted_count,eandm_rejected_count,eandm_added_count,clean_suggested,clean_final,accepted,rejected,added
0,8235901,2023-07-06 11:00:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095114,85610,"{85610=[QW], 1036F=[]}",NaN,{},99212,0,0,0,,,0,1,0
1,8256124,2023-07-06 13:30:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8095124,85610,"{1170F=[], 1160F=[], 85610=[QW], 1159F=[], 103...",NaN,{},99213,0,0,0,,,0,1,0
2,8256117,2023-07-06 13:15:00,*Nurse Visit 15,Office or Other Outpatient Visit,3038,8803379,85610,"{85610=[QW], 1035F=[], 1034F=[]}",NaN,{},99212,0,0,0,,,0,1,0
3,8229493,2023-07-05 14:00:00,*Hosp Follow-up 30,Office or Other Outpatient Visit,3038,8057412,"3008F, 93040, 3077F, 1036F, 3078F","{3008F=[], 93040=[], 3077F=[], 1036F=[], 3078F...",99214,{99214=[25]},99214,1,0,0,99214,99214,1,0,0
4,8234612,2023-07-06 09:30:00,*Follow-up 15,Office or Other Outpatient Visit,3038,8057430,"1160F, 93040, 1159F, 1126F, 1036F","{1160F=[], 93040=[], 1159F=[], 1126F=[], 1036F...",99214,{99214=[25]},99214,1,0,0,99214,99214,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8414,8228747,2023-07-05 14:00:00,*Follow-up 15,OV,1523,4060723,NaN,{81001=[]},99213,{99213=[]},99213,1,0,0,99213,99213,1,0,0
8415,8082306,2023-06-20 13:00:00,uroflow/PVR,OV,1523,4329261,"1160F, 1159F, 1036F","{1160F=[], 1159F=[], 1036F=[]}",99214,{99214=[25]},99214,1,0,0,99214,99214,1,0,0
8416,8213382,2023-07-05 13:30:00,ER FOLLOW-UP,OV,1523,4351334,"1160F, 1159F, 1036F","{3008F=[], 1160F=[], 1159F=[], 87210=[QW], 103...",99213,{99213=[]},99213,1,0,0,99213,99213,1,0,0
8417,8113030,2023-06-30 11:30:00,ESTABLISHED PATIENT,OV,1523,4349846,81025,{81025=[]},99213,{99213=[]},99213,1,0,0,99213,99213,1,0,0


In [158]:
new_df.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/after_applying_0502F_and_99024_rule.xlsx')

In [ ]:
new_df.to_excel('')